In [ ]:
# this is to find problem pics

In [1]:
import os, argparse
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from pytz import timezone
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.backends.cudnn as cudnn
import torchvision
import wandb

import datasets
# from utils import select_device, natural_keys, gazeto3d, angular, getArch
from utils import select_device, natural_keys, gazeto3d, angular, getArch
from model import L2CS

In [2]:
wandb.init(project="smode_rdata")

wandb: Currently logged in as: synthesis-ai (use `wandb login --relogin` to force relogin)


In [2]:
# check if we have the correct number of checkpoint files 
ppath ='/project/results/output/snapshots/' 
for fold in range(15):
    foldstr = f"fold{fold:0>2}"
    cpath =os.path.join(ppath, foldstr)
    files = os.listdir(cpath)
    print(len(files), end=" ")

61 61 61 61 61 61 61 61 61 61 61 61 61 61 61 

In [4]:
def parse_args():
    """Parse input arguments."""
    parser = argparse.ArgumentParser(
        description='Gaze estimation using L2CSNet .')
     # Gaze360
    parser.add_argument(
        '--gaze360image_dir', dest='gaze360image_dir', help='Directory path for gaze images.',
        default='datasets/Gaze360/Image', type=str)
    parser.add_argument(
        '--gaze360label_dir', dest='gaze360label_dir', help='Directory path for gaze labels.',
        default='datasets/Gaze360/Label/test.label', type=str)
    # mpiigaze
    parser.add_argument(
        '--gazeMpiimage_dir', dest='gazeMpiimage_dir', help='Directory path for gaze images.',
        default='datasets/MPIIFaceGaze/Image', type=str)
    parser.add_argument(
        '--gazeMpiilabel_dir', dest='gazeMpiilabel_dir', help='Directory path for gaze labels.',
        default='datasets/MPIIFaceGaze/Label', type=str)
    # Important args -------------------------------------------------------------------------------------------------------
    # ----------------------------------------------------------------------------------------------------------------------
    parser.add_argument(
        '--dataset', dest='dataset', help='gaze360, mpiigaze',
        default= "gaze360", type=str)
    parser.add_argument(
        '--snapshot', dest='snapshot', help='Path to the folder contains models.', 
        default='output/snapshots/L2CS-gaze360-_loader-180-4-lr', type=str)
    parser.add_argument(
        '--evalpath', dest='evalpath', help='path for the output evaluating gaze test.',
        default="evaluation/L2CS-gaze360-_loader-180-4-lr", type=str)
    parser.add_argument(
        '--gpu',dest='gpu_id', help='GPU device id to use [0]',
        default="0", type=str)
    parser.add_argument(
        '--batch_size', dest='batch_size', help='Batch size.',
        default=100, type=int)
    parser.add_argument(
        '--arch', dest='arch', help='Network architecture, can be: ResNet18, ResNet34, [ResNet50], ''ResNet101, ResNet152, Squeezenet_1_0, Squeezenet_1_1, MobileNetV2',
        default='ResNet50', type=str)
    # ---------------------------------------------------------------------------------------------------------------------
    # Important args ------------------------------------------------------------------------------------------------------
    args = parser.parse_args()
    return args

In [3]:
class Nothing:
    pass
args = Nothing()
args.gazeMpiimage_dir = '/project/data/sdata/Image'  #real data 
args.gazeMpiilabel_dir = '/project/data/sdata/Label'  #real label
args.output = '/project/results/output/snapshots/'  #real data model
args.dataset = 'mpiigaze'
args.snapshot='/project/results/output/snapshots/'  #real data snapshots
args.evalpath = '/project/results/rsoutput/evaluation/'
args.gpu_id = '0,1,2,3'
args.gpu_id = '0'
args.batch_size = 20
args.arch = 'ResNet50'
args.bins=28
args.angle = 180
args.bin_width = 4


In [4]:
batch_size=args.batch_size
arch=args.arch
data_set=args.dataset
evalpath =args.evalpath
snapshot_path = args.snapshot
bins=args.bins
angle=args.angle
bin_width=args.bin_width

In [5]:
# args = parse_args()
cudnn.enabled = True
gpu = select_device(args.gpu_id, batch_size=args.batch_size)
transformations = transforms.Compose([
    transforms.Resize(448),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

model_used=getArch(arch, bins)  #resnet50 and 28 bins

In [8]:
# fold=2
# folder = os.listdir(os.path.join(snapshot_path, "fold" + f'{fold:0>2}'))
# folder.sort(key=natural_keys)
# folder.pop(-1)  #remove the tensorboard file
# # print(folder)
# epochs = folder[3]
# os.path.join(snapshot_path+"fold"+f'{fold:0>2}', epochs)

In [9]:
# print(evalpath, snapshot_path)

In [10]:
# print(data_set)

In [11]:
%%time
all_MAE = []
# tosave={}
for fold in range(15):
    print(f"{fold={fold}")
    epoch_values=[]
    mae_values=[]
    
    now = datetime.utcnow()
    now = now.astimezone(timezone('US/Pacific'))
    date_format='%m/%d/%Y %H:%M:%S'
    now = now.strftime(date_format)
    
    print(args.gazeMpiilabel_dir)
    folder = os.listdir(args.gazeMpiilabel_dir)
    folder.sort()  #individual label files
    testlabelpathcombined = [os.path.join(args.gazeMpiilabel_dir, j) for j in folder] 
    
    gaze_dataset=datasets.Mpiigaze(testlabelpathcombined, args.gazeMpiimage_dir, transformations, False, angle, fold)

    test_loader = torch.utils.data.DataLoader(
        dataset=gaze_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4,
        pin_memory=True)

    fold_path = os.path.join(evalpath, 'fold' + f'{fold:0>2}'+'/')  #for evaluation
#     print(f"fold_path is {fold_path}")
    if not os.path.exists(fold_path):
        os.makedirs(fold_path)

#     if not os.path.exists(os.path.join(evalpath, f"fold"+str(fold))):
#         os.makedirs(os.path.join(evalpath, f"fold"+str(fold)))

    # list all epoch for testing
    folder = os.listdir(os.path.join(snapshot_path, "fold" + f'{fold:0>2}'))
    folder.sort(key=natural_keys)
    folder.pop(-1)  #remove the tensorboard file, now all snapshot files
#     print(f"folder={folder}")
                    

    softmax = nn.Softmax(dim=1)
#     with open(os.path.join(evalpath, os.path.join("fold"+f'{fold:0>2}', data_set+".log")), 'w') as outfile:
        
    configuration = (f"\ntest configuration equal gpu_id={gpu}, batch_size={batch_size}, model_arch={arch}\n"
                     f"Start testing dataset={data_set}, FOLD={fold} --{now}---------")
#         print(configuration)
#     tosave['time':now]
#     outfile.write(configuration)
    epoch_list=[]
    avg_MAE=[]
    for epoch in folder: 
        print(f"entering epoch={epoch}")
        model=model_used
        checkpoint = torch.load(os.path.join(snapshot_path+"fold"+f'{fold:0>2}', epoch))
        saved_state_dict = checkpoint['model_state_dict']
        model= nn.DataParallel(model,device_ids=[0])
        model.load_state_dict(saved_state_dict)
        model.cuda(gpu)
        model.eval()
        total = 0
        idx_tensor = [idx for idx in range(28)]
        idx_tensor = torch.FloatTensor(idx_tensor).cuda(gpu)
        avg_error = .0
        with torch.no_grad():
            for j, (images, labels, cont_labels, name) in enumerate(test_loader):
                images = Variable(images).cuda(gpu)
                total += cont_labels.size(0)

                label_pitch = cont_labels[:,0].float()*np.pi/180
                label_yaw = cont_labels[:,1].float()*np.pi/180

                gaze_pitch, gaze_yaw = model(images)

                # Binned predictions
                _, pitch_bpred = torch.max(gaze_pitch.data, 1)
                _, yaw_bpred = torch.max(gaze_yaw.data, 1)

                # Continuous predictions
                pitch_predicted = softmax(gaze_pitch)
                yaw_predicted = softmax(gaze_yaw)

                # mapping from binned (0 to 28) to angels (-42 to 42)                
                pitch_predicted = \
                    torch.sum(pitch_predicted * idx_tensor, 1).cpu() * 3 - 42
                yaw_predicted = \
                    torch.sum(yaw_predicted * idx_tensor, 1).cpu() * 3 - 42

                pitch_predicted = pitch_predicted*np.pi/180
                yaw_predicted = yaw_predicted*np.pi/180

                for p,y,pl,yl in zip(pitch_predicted, yaw_predicted, label_pitch, label_yaw):
                    avg_error += angular(gazeto3d([p,y]), gazeto3d([pl,yl]))

        x = ''.join(filter(lambda i: i.isdigit(), epoch))
#             print(f"x={x}")
        epoch_list.append(x)
        mean_mae = avg_error/total  #mean mae over the 3000 iamges
        avg_MAE.append(mean_mae)  
#             print(f"total={total}")
        
        loger = f"[{epoch}---{args.dataset}] Total Num:{total},MAE:{mean_mae}"
#         outfile.write(loger)
        print(loger)
#             print(f"done epoch={epochs}")
        epochn = int(x)
#         wandb.log({'epoch':epochn,'total':total, 'MAE':mean_mae, 'FOLD':fold })
#         tosave ={'epoch':epochn,'total':total, 'MAE':mean_mae, 'FOLD':fold }
        epoch_values.append(epochn)
        mae_values.append(mean_mae)
# #         wandb.log(tosave)
    all_MAE.append(avg_MAE)
    data = [[x, y] for (x, y) in zip(epoch_values, mae_values)]
    table = wandb.Table(data=data, columns = ["x", "y"])
    wandb.log({"epoch_losses" : wandb.plot.line(table, "x", "y",
               title="Epoch loss for each fold")})
    
#     fig = plt.figure(figsize=(14, 8))        
#     plt.xlabel('epoch')
#     plt.ylabel('avg')
#     plt.title('Gaze angular error')
#     plt.legend()
#     plt.plot(epoch_list, avg_MAE, color='k', label='mae')
#     fig.savefig(os.path.join(evalpath, os.path.join("fold"+f'{fold:0>2}', data_set+".png")), format='png')
#     # plt.show() 

/project/data/Label
0 items removed from dataset that have an angle > 180
entering epoch=epoch_1.pkl
[epoch_1.pkl---mpiigaze] Total Num:3000,MAE:13.443461294262843
entering epoch=epoch_2.pkl
[epoch_2.pkl---mpiigaze] Total Num:3000,MAE:13.272026728035604
entering epoch=epoch_3.pkl
[epoch_3.pkl---mpiigaze] Total Num:3000,MAE:13.05084913579883
entering epoch=epoch_4.pkl
[epoch_4.pkl---mpiigaze] Total Num:3000,MAE:13.215686498409612
entering epoch=epoch_5.pkl
[epoch_5.pkl---mpiigaze] Total Num:3000,MAE:14.751142108376152
entering epoch=epoch_6.pkl
[epoch_6.pkl---mpiigaze] Total Num:3000,MAE:14.505299525619902
entering epoch=epoch_7.pkl
[epoch_7.pkl---mpiigaze] Total Num:3000,MAE:13.462039524435633
entering epoch=epoch_8.pkl
[epoch_8.pkl---mpiigaze] Total Num:3000,MAE:12.960322281379447
entering epoch=epoch_9.pkl
[epoch_9.pkl---mpiigaze] Total Num:3000,MAE:14.295336065145978
entering epoch=epoch_10.pkl
[epoch_10.pkl---mpiigaze] Total Num:3000,MAE:14.111884437230035
entering epoch=epoch_11.p

[epoch_36.pkl---mpiigaze] Total Num:3000,MAE:12.405906232419197
entering epoch=epoch_37.pkl
[epoch_37.pkl---mpiigaze] Total Num:3000,MAE:13.12468012015193
entering epoch=epoch_38.pkl
[epoch_38.pkl---mpiigaze] Total Num:3000,MAE:12.36502243007355
entering epoch=epoch_39.pkl
[epoch_39.pkl---mpiigaze] Total Num:3000,MAE:12.129889539532288
entering epoch=epoch_40.pkl
[epoch_40.pkl---mpiigaze] Total Num:3000,MAE:13.006008781154232
entering epoch=epoch_41.pkl
[epoch_41.pkl---mpiigaze] Total Num:3000,MAE:12.14798033160813
entering epoch=epoch_42.pkl
[epoch_42.pkl---mpiigaze] Total Num:3000,MAE:13.112901735668938
entering epoch=epoch_43.pkl
[epoch_43.pkl---mpiigaze] Total Num:3000,MAE:12.447884594604478
entering epoch=epoch_44.pkl
[epoch_44.pkl---mpiigaze] Total Num:3000,MAE:12.855781541459212
entering epoch=epoch_45.pkl
[epoch_45.pkl---mpiigaze] Total Num:3000,MAE:13.045245425648451
entering epoch=epoch_46.pkl
[epoch_46.pkl---mpiigaze] Total Num:3000,MAE:13.506203148067017
entering epoch=epoc

[epoch_4.pkl---mpiigaze] Total Num:3000,MAE:15.327281892632193
entering epoch=epoch_5.pkl
[epoch_12.pkl---mpiigaze] Total Num:3000,MAE:16.793003724939275
entering epoch=epoch_13.pkl
[epoch_13.pkl---mpiigaze] Total Num:3000,MAE:15.606346300982707
entering epoch=epoch_14.pkl
[epoch_14.pkl---mpiigaze] Total Num:3000,MAE:15.119389624180405
entering epoch=epoch_15.pkl
[epoch_15.pkl---mpiigaze] Total Num:3000,MAE:15.636851413767198
entering epoch=epoch_16.pkl
[epoch_16.pkl---mpiigaze] Total Num:3000,MAE:16.3140124059663
entering epoch=epoch_17.pkl
[epoch_17.pkl---mpiigaze] Total Num:3000,MAE:16.146819446768184
entering epoch=epoch_18.pkl
[epoch_18.pkl---mpiigaze] Total Num:3000,MAE:16.295863933331184
entering epoch=epoch_19.pkl
[epoch_19.pkl---mpiigaze] Total Num:3000,MAE:16.1025489243627
entering epoch=epoch_20.pkl
[epoch_20.pkl---mpiigaze] Total Num:3000,MAE:16.264760520692807
entering epoch=epoch_21.pkl
[epoch_21.pkl---mpiigaze] Total Num:3000,MAE:16.180656471924838
entering epoch=epoch_2

[epoch_46.pkl---mpiigaze] Total Num:3000,MAE:14.791029315318864
entering epoch=epoch_47.pkl
[epoch_47.pkl---mpiigaze] Total Num:3000,MAE:14.38460914620323
entering epoch=epoch_48.pkl
[epoch_48.pkl---mpiigaze] Total Num:3000,MAE:14.8976672786169
entering epoch=epoch_49.pkl
[epoch_49.pkl---mpiigaze] Total Num:3000,MAE:14.658377313357686
entering epoch=epoch_50.pkl
[epoch_50.pkl---mpiigaze] Total Num:3000,MAE:14.664195020728252
entering epoch=epoch_51.pkl
[epoch_51.pkl---mpiigaze] Total Num:3000,MAE:14.220815957080788
entering epoch=epoch_52.pkl
[epoch_52.pkl---mpiigaze] Total Num:3000,MAE:14.554654071274756
entering epoch=epoch_53.pkl
[epoch_53.pkl---mpiigaze] Total Num:3000,MAE:14.657063695186382
entering epoch=epoch_54.pkl
[epoch_54.pkl---mpiigaze] Total Num:3000,MAE:14.719934317069875
entering epoch=epoch_55.pkl
[epoch_55.pkl---mpiigaze] Total Num:3000,MAE:15.08148838872941
entering epoch=epoch_56.pkl
[epoch_56.pkl---mpiigaze] Total Num:3000,MAE:14.575834969473318
entering epoch=epoch

[epoch_20.pkl---mpiigaze] Total Num:3000,MAE:15.68345740616499
entering epoch=epoch_21.pkl
[epoch_21.pkl---mpiigaze] Total Num:3000,MAE:15.347518899031165
entering epoch=epoch_22.pkl
[epoch_22.pkl---mpiigaze] Total Num:3000,MAE:15.963000541224366
entering epoch=epoch_23.pkl
[epoch_23.pkl---mpiigaze] Total Num:3000,MAE:15.881658775509852
entering epoch=epoch_24.pkl
[epoch_24.pkl---mpiigaze] Total Num:3000,MAE:15.518838449869346
entering epoch=epoch_25.pkl
[epoch_25.pkl---mpiigaze] Total Num:3000,MAE:15.677812643453331
entering epoch=epoch_26.pkl
[epoch_26.pkl---mpiigaze] Total Num:3000,MAE:15.349558176732328
entering epoch=epoch_27.pkl
[epoch_27.pkl---mpiigaze] Total Num:3000,MAE:15.474000973519578
entering epoch=epoch_28.pkl
[epoch_28.pkl---mpiigaze] Total Num:3000,MAE:15.207026891675511
entering epoch=epoch_29.pkl
[epoch_29.pkl---mpiigaze] Total Num:3000,MAE:15.62788400601343
entering epoch=epoch_30.pkl
[epoch_30.pkl---mpiigaze] Total Num:3000,MAE:15.457767080339568
entering epoch=epo

[epoch_49.pkl---mpiigaze] Total Num:3000,MAE:14.007031895880745
entering epoch=epoch_50.pkl
[epoch_50.pkl---mpiigaze] Total Num:3000,MAE:13.354894933252593
entering epoch=epoch_51.pkl
[epoch_51.pkl---mpiigaze] Total Num:3000,MAE:13.546235237391773
entering epoch=epoch_52.pkl
[epoch_52.pkl---mpiigaze] Total Num:3000,MAE:13.583538734283625
entering epoch=epoch_53.pkl
[epoch_53.pkl---mpiigaze] Total Num:3000,MAE:13.427248854184581
entering epoch=epoch_54.pkl
[epoch_54.pkl---mpiigaze] Total Num:3000,MAE:13.744825935843966
entering epoch=epoch_55.pkl
[epoch_55.pkl---mpiigaze] Total Num:3000,MAE:13.534249820090949
entering epoch=epoch_56.pkl
[epoch_56.pkl---mpiigaze] Total Num:3000,MAE:13.306292139460714
entering epoch=epoch_57.pkl
[epoch_57.pkl---mpiigaze] Total Num:3000,MAE:13.888512807284021
entering epoch=epoch_58.pkl
[epoch_58.pkl---mpiigaze] Total Num:3000,MAE:13.728339025250746
entering epoch=epoch_59.pkl
[epoch_59.pkl---mpiigaze] Total Num:3000,MAE:13.757047542549055
entering epoch=e

[epoch_18.pkl---mpiigaze] Total Num:3000,MAE:15.421412457018587
entering epoch=epoch_19.pkl
[epoch_19.pkl---mpiigaze] Total Num:3000,MAE:15.036133070483512
entering epoch=epoch_20.pkl
[epoch_20.pkl---mpiigaze] Total Num:3000,MAE:16.663133846445962
entering epoch=epoch_21.pkl
[epoch_21.pkl---mpiigaze] Total Num:3000,MAE:15.752842020451666
entering epoch=epoch_22.pkl
[epoch_22.pkl---mpiigaze] Total Num:3000,MAE:15.106725304690768
entering epoch=epoch_23.pkl
[epoch_23.pkl---mpiigaze] Total Num:3000,MAE:15.27131628777592
entering epoch=epoch_24.pkl
[epoch_24.pkl---mpiigaze] Total Num:3000,MAE:15.334627959275638
entering epoch=epoch_25.pkl
[epoch_25.pkl---mpiigaze] Total Num:3000,MAE:15.883660688116183
entering epoch=epoch_26.pkl
[epoch_26.pkl---mpiigaze] Total Num:3000,MAE:15.979766077582127
entering epoch=epoch_27.pkl
[epoch_27.pkl---mpiigaze] Total Num:3000,MAE:14.90860553137134
entering epoch=epoch_28.pkl
[epoch_28.pkl---mpiigaze] Total Num:3000,MAE:15.573748181415409
entering epoch=epo

[epoch_47.pkl---mpiigaze] Total Num:3000,MAE:12.975422288931512
entering epoch=epoch_48.pkl
[epoch_48.pkl---mpiigaze] Total Num:3000,MAE:12.587137106404096
entering epoch=epoch_49.pkl
[epoch_49.pkl---mpiigaze] Total Num:3000,MAE:13.459087768287112
entering epoch=epoch_50.pkl
[epoch_50.pkl---mpiigaze] Total Num:3000,MAE:13.097324676105542
entering epoch=epoch_51.pkl
[epoch_51.pkl---mpiigaze] Total Num:3000,MAE:13.236777913324929
entering epoch=epoch_52.pkl
[epoch_52.pkl---mpiigaze] Total Num:3000,MAE:13.449206031117477
entering epoch=epoch_53.pkl
[epoch_53.pkl---mpiigaze] Total Num:3000,MAE:13.203492093179014
entering epoch=epoch_54.pkl
[epoch_54.pkl---mpiigaze] Total Num:3000,MAE:13.03241461894205
entering epoch=epoch_55.pkl
[epoch_55.pkl---mpiigaze] Total Num:3000,MAE:13.276500459788325
entering epoch=epoch_56.pkl
[epoch_56.pkl---mpiigaze] Total Num:3000,MAE:13.367988459407725
entering epoch=epoch_57.pkl
[epoch_57.pkl---mpiigaze] Total Num:3000,MAE:13.634471288038206
entering epoch=ep

[epoch_22.pkl---mpiigaze] Total Num:3000,MAE:13.117064904130153
entering epoch=epoch_23.pkl
[epoch_23.pkl---mpiigaze] Total Num:3000,MAE:13.915942550752279
entering epoch=epoch_24.pkl
[epoch_24.pkl---mpiigaze] Total Num:3000,MAE:12.795620806086367
entering epoch=epoch_25.pkl
[epoch_25.pkl---mpiigaze] Total Num:3000,MAE:12.679764333056012
entering epoch=epoch_26.pkl
[epoch_26.pkl---mpiigaze] Total Num:3000,MAE:12.790046600306919
entering epoch=epoch_27.pkl
[epoch_27.pkl---mpiigaze] Total Num:3000,MAE:14.123887835976774
entering epoch=epoch_28.pkl
[epoch_28.pkl---mpiigaze] Total Num:3000,MAE:13.19609645001601
entering epoch=epoch_29.pkl
[epoch_29.pkl---mpiigaze] Total Num:3000,MAE:13.509032625813406
entering epoch=epoch_30.pkl
[epoch_30.pkl---mpiigaze] Total Num:3000,MAE:12.952924637763283
entering epoch=epoch_31.pkl
[epoch_31.pkl---mpiigaze] Total Num:3000,MAE:13.632001085330822
entering epoch=epoch_32.pkl
[epoch_32.pkl---mpiigaze] Total Num:3000,MAE:14.031429420069461
entering epoch=ep

[epoch_51.pkl---mpiigaze] Total Num:3000,MAE:14.185796356544694
entering epoch=epoch_52.pkl
[epoch_52.pkl---mpiigaze] Total Num:3000,MAE:14.212259423389723
entering epoch=epoch_53.pkl
[epoch_53.pkl---mpiigaze] Total Num:3000,MAE:13.129036779824846
entering epoch=epoch_54.pkl
[epoch_54.pkl---mpiigaze] Total Num:3000,MAE:14.069645980093629
entering epoch=epoch_55.pkl
[epoch_55.pkl---mpiigaze] Total Num:3000,MAE:13.999208543935248
entering epoch=epoch_56.pkl
[epoch_56.pkl---mpiigaze] Total Num:3000,MAE:13.411143334328566
entering epoch=epoch_57.pkl
[epoch_57.pkl---mpiigaze] Total Num:3000,MAE:13.67955570745424
entering epoch=epoch_58.pkl
[epoch_58.pkl---mpiigaze] Total Num:3000,MAE:13.758606504918982
entering epoch=epoch_59.pkl
[epoch_59.pkl---mpiigaze] Total Num:3000,MAE:14.317842373820136
entering epoch=epoch_60.pkl
[epoch_60.pkl---mpiigaze] Total Num:3000,MAE:14.01510901779125
/project/data/Label
0 items removed from dataset that have an angle > 180
entering epoch=epoch_1.pkl
[epoch_1.

In [12]:
all_MAE=np.array(all_MAE)

In [13]:

print(all_MAE.shape)
print(all_MAE.mean(axis=0))
print(all_MAE.mean(axis=1))
print(all_MAE.mean(axis=0).argmin()+1 ,all_MAE.mean(axis=0).min())

(15, 60)
[13.94803401 14.12621199 13.87429765 14.28510734 14.52365314 14.26136898
 14.72937786 14.41138492 14.6085134  14.40903828 14.74414477 14.65334247
 14.47695917 14.42759779 14.6158625  14.64778402 14.70694968 14.33433888
 14.38146837 14.54328737 14.6441113  14.35636946 14.5157564  14.26281949
 14.21101515 14.22714451 14.3784224  14.34982691 14.38313223 14.26448504
 14.34562468 14.23770554 14.3856845  14.26282735 14.37663472 14.16338009
 14.29246139 14.21649813 14.10672231 14.33135179 14.18399642 14.29394944
 14.2081887  14.29561731 14.27500562 14.25953762 14.16549394 14.20877727
 14.35553398 14.11678406 14.37833955 14.31601183 14.09729345 14.28963468
 14.36563507 14.26609289 14.42344366 14.40188922 14.29202992 14.40147921]
[13.54034763 12.79600434 12.64661947 15.66326575 14.79024152 15.01233557
 15.48823844 13.92143397 16.25341114 15.10532141 13.31095936 15.2777245
 13.33585395 13.73430159 14.27779881]
3 13.874297651351478
